In [61]:
# import important library
import pandas as pd
import numpy as np
import bot

In [2]:
import importlib
importlib.reload(bot)

<module 'bot' from '/Users/henrytran/Documents/GitHub/aias_trading25/bot.py'>

# 1. Data Processing

In [3]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [4]:
# convert time strings into timestamps, and take only year, month and date values
daily_df["date"]=pd.to_datetime(daily_df["date"]).dt.strftime("%Y-%m-%d")

In [5]:
daily_df.shape

(2651, 9)

The dataset has 2651 records from 2014 to 2022.

As the data is needed for training and testing, every records before 2020 will be used for training and every records after 2020 will be used for testing.

In [6]:
# splitting the data
training = daily_df[daily_df["date"]<"2020-01-01"].iloc[::-1]
testing = daily_df[daily_df["date"]>="2020-01-01"].iloc[::-1]

In [7]:
# convert the data into csv files
training.to_csv('training.csv')
testing.to_csv('testing.csv')

After splitting, the training dataset has 1860 records and the testing data has 791 records.

In [8]:
print(training.shape)
print(testing.shape)

(1860, 9)
(791, 9)


# 2. Hill-climbing algorithm

## a. Hill-climbing algorithm 

In [173]:
# The bounds will be 
# sma: [(5,50),(51,100)]
# smaema: [(5,50),(51,100),(0,1)]
# weights: [(0,10),(0,10),(0,10),(5,50),(5,50),(5,50),(0,1), (0,10),(0,10),(0,10),(51,100),(51,100),(51,100),(0,1)]

def hill_climbing(bot_type, bounds, max_iter=1000):
    rng = np.random.default_rng()
    high_window = None
    low_window = None
    alpha=0
    new_high_frequency_window = high_window 
    new_low_frequency_window = low_window
    new_alpha=0
    # processing the bounds inputs
    if bot_type.lower() == 'sma':
        high_window = int(rng.integers(bounds[0][0],bounds[0][1]))
        low_window = int(rng.integers(bounds[1][0],bounds[1][1]))
    elif bot_type.lower() == 'smaema':
        high_window = int(rng.integers(bounds[0][0],bounds[0][1]))
        low_window = int(rng.integers(bounds[1][0],bounds[1][1]))
        alpha = rng.uniform(bounds[2][0], bounds[-1][1])
    elif bot_type.lower() == 'complex':
        # generate the values for high
        weight_sma_high= int(rng.integers(bounds[0][0],bounds[0][1]))
        weight_lma_high= int(rng.integers(bounds[1][0],bounds[0][1]))
        weight_ema_high= int(rng.integers(bounds[2][0],bounds[0][1]))
        window_sma_high= int(rng.integers(bounds[3][0],bounds[0][1]))
        window_lma_high= int(rng.integers(bounds[4][0],bounds[0][1]))
        window_ema_high= int(rng.integers(bounds[5][0],bounds[0][1]))
        alpha_high = rng.uniform(bounds[6][0], bounds[6][1])
        high_window = [weight_sma_high,weight_lma_high, weight_ema_high, window_sma_high, window_lma_high, window_ema_high, alpha_high]
        # generate the values for low
        weight_sma_low= int(rng.integers(bounds[7][0],bounds[7][1]))
        weight_lma_low= int(rng.integers(bounds[8][0],bounds[8][1]))
        weight_ema_low= int(rng.integers(bounds[9][0],bounds[9][1]))
        window_sma_low= int(rng.integers(bounds[10][0],bounds[10][1]))
        window_lma_low= int(rng.integers(bounds[11][0],bounds[11][1]))
        window_ema_low= int(rng.integers(bounds[12][0],bounds[12][1]))
        alpha_low = rng.uniform(bounds[13][0], bounds[13][1])
        low_window = [weight_sma_low, weight_lma_low, weight_ema_low, window_sma_low, window_lma_low, window_ema_low, alpha_low]

    # the best cash
    cash1=0
    for i in range(max_iter):
        # find the correct parameter tweak:
        if bot_type.lower() == 'sma':
            new_high_frequency_window, new_low_frequency_window = window_tweak(high_window, low_window, bounds[0], bounds[1])
        elif bot_type.lower() == 'smaema':
            new_high_frequency_window, new_low_frequency_window = window_tweak(high_window, low_window, bounds[0], bounds[1])
            new_alpha = alpha_tweak(alpha, bounds[2])
        elif bot_type.lower() == 'complex':
            # generate the values 
            new_high_frequency_window, new_low_frequency_window = complex_tweak(high_window, low_window, bounds)
            
        # run the total cash return after trading
        cash1=bot_fitness_func(bot_type, high_window, low_window, alpha)
        cash2=bot_fitness_func(bot_type, new_high_frequency_window, new_low_frequency_window,alpha)
        # compare the cash earned after tweaking the parameters
        if cash2 > cash1:
            high_window = new_high_frequency_window
            low_window = new_low_frequency_window
            alpha=new_alpha
            cash1 = cash2
    if bot_type.lower() == 'sma' or bot_type.lower() == 'complex':
        return [high_window, low_window], float(cash1)
    elif bot_type.lower() == 'smaema':
        return [high_window, low_window, alpha], float(cash1)

def window_tweak(hfw, lfw, window_range1, window_range2):
    rng = np.random.default_rng()
    for _ in range(100):
        # new high frequency window
        a=int(rng.integers(-5,6))
        new_hfw=hfw+a
        # new low frequency window
        b=int(rng.integers(-5,6))
        new_lfw=lfw+b
        # we check to make sure that new_hfw in range(11,40) and new_lfw(2,10)
        new_hfw = max(window_range1[0], min(window_range1[1], new_hfw)) # can change the bounds to 5-50 
        new_lfw = max(window_range2[0], min(window_range2[1], new_lfw)) # can change the bounds to 51-100
        if new_hfw < new_lfw:
            return new_hfw, new_lfw
    return hfw, lfw

def alpha_tweak(alpha, alpha_range):
    rng = np.random.default_rng()
    for _ in range(100):
        diff = rng.uniform(-0.15, 0.15)
        new_alpha = alpha + diff
        if alpha_range[0]<= new_alpha <= alpha_range[1]:
            return new_alpha
    return alpha # if after 100 loops and cannot find the optimal value, then we return the original alpha

def weights_tweak(weight_lst, weight_range):
    rng = np.random.default_rng()
    weights=[]
    for i in range(len(weight_range)):
        weight = weight_lst[i]
        diff = rng.uniform(-1.5, 1.5)
        weight += diff
        new_weight = max(weight_range[i][0], min(weight_range[i][1], weight))
        weights.append(new_weight)
    return weights


def complex_tweak(hfw, lfw, bounds):
    # in this code, hfw and lfw input are array-type in format of [w1_h, w2_h, w3_h, d1_h, d2_h, d3_h, sf_h, w1_l, w2_l, w3_l, d1_l, d2_l, d3_l, sf_l]
    # weight tweak

    new_hfw_w1, new_hfw_w2,new_hfw_w3 = weights_tweak(hfw[0:3], bounds[0:3])
    new_lfw_w1, new_lfw_w2,new_lfw_w3 = weights_tweak(lfw[0:3], bounds[7:10])

    # window tweak
    new_hfw_d1, new_lfw_d1 = window_tweak(hfw[3], lfw[3], bounds[3], bounds[10])
    new_hfw_d2, new_lfw_d2 = window_tweak(hfw[4], lfw[4], bounds[4], bounds[11])
    new_hfw_d3, new_lfw_d3 = window_tweak(hfw[5], lfw[5], bounds[5], bounds[12])

    # alpha tweak
    new_hfw_alpha = alpha_tweak(hfw[-1], bounds[6])
    new_lfw_alpha = alpha_tweak(lfw[-1], bounds[13])

    return [new_hfw_w1, new_hfw_w2, new_hfw_w3, new_hfw_d1, new_hfw_d2, new_hfw_d3,new_hfw_alpha], [new_lfw_w1, new_lfw_w2, new_lfw_w3, new_lfw_d1, new_lfw_d2, new_lfw_d3, new_lfw_alpha]


## b. Fitness function on training and testing data

In [172]:
def bot_training(bot_type, high_window, low_window, alpha): # change bot_signals to high_frequency_window_size, low_frequency_window_size
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    bot_signals=[]
    close_price = pd.read_csv('training.csv')['close']
    
    if bot_type.lower() == 'sma':
        bot_signals = bot.get_signals_sma2(close_price, high_window, low_window) #need to think about how to call 2 other bot algorithms
    elif bot_type.lower() == 'smaema':
        bot_signals = bot.get_signals_smaema(close_price, high_window, low_window, alpha)
    elif bot_type.lower() == 'complex': #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, high_window, low_window)

    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(min(len(bot_signals),len(close_price)-1)):
        close=close_price.iloc[i]
        # buy, ensure we have cash to buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
        # sell, ensure we have bitcoin to sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
        return cash
    elif cash >0:
        return cash

In [189]:
def bot_testing(bot_type, optimal_values):
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    bot_signals=[]
    close_price = pd.read_csv('testing.csv')['close']
    time = pd.read_csv('testing.csv')['date']
    result=[]

    # intialise bot, use training dataset for optimisation algorithms fitness functions
    if bot_type.lower() == 'sma':
        bot_signals = bot.get_signals_sma2(close_price, optimal_values[0], optimal_values[1])
    elif bot_type.lower() == 'smaema':
        bot_signals = bot.get_signals_smaema(close_price, optimal_values[0], optimal_values[1], optimal_values[2])
    elif bot_type.lower() == 'complex': #as alpha values were stored in the list of high_window and low_window
        bot_signals = bot.get_signals_complex(close_price, optimal_values[0], optimal_values[1])
    
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0
    
    #loop through the time length
    for i in range(min(len(bot_signals),len(close_price)-1)):
        close=close_price[i]
        # buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
        result.append([time.iloc[-1],cash, bitcoin])
        return result
    elif cash>0:
        result.append([time.iloc[-1],cash, bitcoin])
        return result

In [164]:
def bot_evaluation(bot_type, optimal_values):
    # This function will returns the result nicely
    result_lst=bot_testing(bot_type, optimal_values)
    result_df=pd.DataFrame(result_lst, columns=["Time", "Cash", "Bitcoin"])
    print(result_df.to_string(index=False, justify="center", float_format='{:,.2f}'.format))
    #return result_df

# 3. Running the whole bot
## a. Running group A (H: 1-10, L: 11-40)
### PART A: Training the bot to get the optimal result

In [177]:
#sma
sma_optimals, cash_result = hill_climbing('sma',[(1,10),(11,40)])
sma_optimals, cash_result

([9, 25], 5965.086841833178)

In [178]:
#smaema
samema_optimals, cash_result = hill_climbing('smaema', [(1,10),(11,40),(0,1)])
samema_optimals, cash_result

([9, 40, 0.9674799395055566], 1109.7384627175877)

In [179]:
#complex
complex_optimals, cash_result = hill_climbing('complex',[(0,10),(0,10),(0,10),(1,10),(1,10),(1,10),(0,1), (0,10),(0,10),(0,10),(11,40),(11,40),(11,40),(0,1)])
complex_optimals, cash_result

([[1.1659219982350497,
   4.296358694985838,
   2.8518464703241895,
   10,
   10,
   10,
   0.8177740138150238],
  [10, 4.743888529169307, 8.896757425371685, 34, 29, 20, 0.08480724374063864]],
 36215.514177773526)

### PART B: Testing the bot with optimal results


In [180]:
#sma - nice printout
bot_evaluation('sma', sma_optimals)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-02-24 1,265.15   0.00  
2020-03-31     0.00   0.19  
2020-05-26 1,638.97   0.00  
2020-06-02     0.00   0.17  
2020-06-18 1,517.67   0.00  
2020-07-13     0.00   0.16  
2020-08-27 1,752.28   0.00  
2020-09-22     0.00   0.16  
2020-10-06 1,659.99   0.00  
2020-10-10     0.00   0.14  
2021-01-23 4,438.80   0.00  
2021-02-05     0.00   0.11  
2021-03-02 5,287.76   0.00  
2021-03-09     0.00   0.09  
2021-03-28 5,053.32   0.00  
2021-04-03     0.00   0.09  
2021-04-22 4,306.36   0.00  
2021-05-07     0.00   0.07  
2021-05-15 3,298.22   0.00  
2021-06-15     0.00   0.08  
2021-06-23 2,599.68   0.00  
2021-07-27     0.00   0.06  
2021-09-12 2,852.50   0.00  
2021-10-05     0.00   0.05  
2021-11-19 3,029.29   0.00  
2021-12-27     0.00   0.06  
2022-01-03 2,609.83   0.00  
2022-02-07     0.00   0.06  
2022-02-22 2,142.18   0.00  
2022-03-01 2,142.18   0.00  


In [190]:
#smaema
bot_evaluation('smaema', samema_optimals)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-03-14   677.54   0.00  
2020-04-29     0.00   0.07  
2020-09-06   744.10   0.00  
2020-09-07     0.00   0.07  
2020-09-08   683.04   0.00  
2020-09-11     0.00   0.06  
2020-09-13   639.07   0.00  
2020-09-14     0.00   0.06  
2020-09-23   576.42   0.00  
2020-09-24     0.00   0.05  
2020-10-02   534.00   0.00  
2020-10-05     0.00   0.05  
2020-10-06   493.66   0.00  
2020-10-08     0.00   0.04  
2021-04-24 2,130.80   0.00  
2021-04-26     0.00   0.04  
2021-05-12 1,831.10   0.00  
2021-07-29     0.00   0.04  
2021-09-28 1,766.40   0.00  
2021-09-30     0.00   0.04  
2021-12-03 2,037.73   0.00  
2022-03-01 2,037.73   0.00  


In [182]:
#complex
bot_evaluation('complex',complex_optimals)

   Time      Cash    Bitcoin
2020-03-24     0.00   0.14  
2021-05-21 5,189.91   0.00  
2021-06-01     0.00   0.14  
2022-03-01 5,747.18   0.00  


## b. Running group B (H: 5-50, L: 51-100)
### PART A: Training the bot to get the optimal result

In [183]:
#sma
sma_optimals, cash_result = hill_climbing('sma',[(5,50),(51,100)])
sma_optimals, cash_result

([17, 100], 18743.5259647998)

In [184]:
#smaema
samema_optimals, cash_result = hill_climbing('smaema', [(5,50),(51,100),(0,1)])
samema_optimals, cash_result

([45, 92, 0.5971497672955902], 8294.450860676487)

In [185]:
#complex
complex_optimals, cash_result = hill_climbing('complex',[(0,10),(0,10),(0,10),(5,50),(5,50),(5,50),(0,1), (0,10),(0,10),(0,10),(51,100),(51,100),(51,100),(0,1)])
complex_optimals, cash_result

([[0, 8.334288659563903, 1.6274332041189186, 16, 5, 11, 0.47868025623396393],
  [6.260524068932414, 0, 2.194494465269502, 51, 97, 80, 0.2384507520431139]],
 28280.00953232217)

### PART B: Testing the bot with optimal results

In [186]:
#sma - nice printout
bot_evaluation('sma', sma_optimals)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-03-27   835.70   0.00  
2020-05-06     0.00   0.09  
2021-05-17 3,744.44   0.00  
2021-08-08     0.00   0.08  
2021-12-09 3,823.16   0.00  
2022-03-01 3,823.16   0.00  


In [191]:
#smaema
bot_evaluation('smaema', samema_optimals)

   Time      Cash    Bitcoin
2020-01-01     0.00   0.14  
2020-03-14   677.54   0.00  
2020-04-29     0.00   0.07  
2020-09-06   744.10   0.00  
2020-09-07     0.00   0.07  
2020-09-08   683.04   0.00  
2020-09-11     0.00   0.06  
2020-09-13   639.07   0.00  
2020-09-14     0.00   0.06  
2020-09-23   576.42   0.00  
2020-09-24     0.00   0.05  
2020-10-02   534.00   0.00  
2020-10-05     0.00   0.05  
2020-10-06   493.66   0.00  
2020-10-08     0.00   0.04  
2021-04-24 2,130.80   0.00  
2021-04-26     0.00   0.04  
2021-05-12 1,831.10   0.00  
2021-07-29     0.00   0.04  
2021-09-28 1,766.40   0.00  
2021-09-30     0.00   0.04  
2021-12-03 2,037.73   0.00  
2022-03-01 2,037.73   0.00  


In [188]:
#complex
bot_evaluation('complex',complex_optimals)

   Time      Cash    Bitcoin
2020-03-23     0.00   0.15  
2021-05-21 5,409.81   0.00  
2021-05-27     0.00   0.14  
2021-05-29 4,578.30   0.00  
2021-06-02     0.00   0.12  
2022-03-01 4,949.03   0.00  
